In [1]:
import gurobipy as gp
import pandas as pd
import geopandas as gpd
from gurobipy import Model, GRB, quicksum
from pyproj import Transformer
from openrouteservice import Client
import numpy as np
import pickle
import os

In [2]:
# Chave de cliente do OpenRouteService
client_ors = Client(key='eyJvcmciOiI1YjNjZTM1OTc4NTExMTAwMDFjZjYyNDgiLCJpZCI6ImU4MGRiYmE2OTVlODQwMmY5MWVjMDE2ZTY5MDcyMmMwIiwiaCI6Im11cm11cjY0In0=')

In [3]:
demanda_df = pd.read_csv('Downloads/SE_Municipios(se_internacoes_2024)(in).csv', sep=';')
capacidade_df = pd.read_csv('Downloads/SE_coords_medias.csv')
alvo_df = pd.read_csv('Downloads/SE_Municipios_dv.csv', sep=';')

In [4]:
hospitais_df = pd.read_csv('Downloads/cnes_cnv_estabse_115.csv', sep=';')

In [5]:
# Conjuntos
I = demanda_df['CD_MUN'].tolist() # Municípios
J = alvo_df['CD_MUN'].tolist() # Municípios candidatos
H_i = demanda_df.set_index('CD_MUN')['Internacoes_Set/2024-Ago/2025'].to_dict() # Dicionário de Demanda (Di)
K_i = demanda_df.set_index('CD_MUN')['Qtd_leitos'].to_dict() # dicionário de capacidade (leitos)

In [6]:
# Cálculo do total de dias de internação
## Multiplica as duas colunas e cria uma nova com o total de dias de internação
demanda_df['Média_permanência'] = pd.to_numeric(
    demanda_df['Média_permanência'].str.replace(',', '.'), errors='coerce'
)
## Calcula a média apenas dos valores positivos e não nulos
media_geral_permanencia = demanda_df.loc[
    demanda_df['Média_permanência'] > 0, 'Média_permanência'
].mean()

print(f"Média geral usada para substituir zeros: {media_geral_permanencia:.2f} dias")

## Substitui 0 ou NaN pela média geral
demanda_df['Média_permanência_ajustada'] = demanda_df['Média_permanência'].replace(0, np.nan)
demanda_df.fillna({'Média_permanência_ajustada': media_geral_permanencia}, inplace=True)

## Agora calcula os dias de internação totais corretamente
demanda_df['total_dias_internacao'] = (
    demanda_df['Média_permanência_ajustada'] * demanda_df['Internacoes_Set/2024-Ago/2025']
)

Média geral usada para substituir zeros: 3.79 dias


In [7]:
# Transformação de registro de dados de referência espacial
transformer = Transformer.from_crs("EPSG:4674", "EPSG:4326", always_xy=True)

## Aplicar transformação
capacidade_df['lon'], capacidade_df['lat'] = transformer.transform(capacidade_df['lon'].values, capacidade_df['lat'].values)

print(capacidade_df[['CD_MUN','lon','lat']])

     CD_MUN        lon        lat
0   2801702 -37.750072 -11.478330
1   2803609 -37.157668 -10.798400
2   2800605 -37.020106 -10.901967
3   2807303 -36.877770 -10.204712
4   2806206 -37.482709 -11.031149
..      ...        ...        ...
70  2805802 -37.774542 -11.066061
71  2806107 -37.036196 -10.692449
72  2807105 -37.822284 -10.748819
73  2806404 -36.617888 -10.281743
74  2801405 -37.701177 -10.360228

[75 rows x 3 columns]


In [8]:
# Colunas esperadas: 'CD_MUN', 'lon', 'lat' (em ambos)
origens_codes = capacidade_df['CD_MUN'].tolist()
alvos_codes = alvo_df['CD_MUN'].tolist()

# Lista de coordenadas: [lon, lat]
origens_locs = capacidade_df[['lon', 'lat']].values.tolist()
alvos_locs = alvo_df[['lon', 'lat']].values.tolist()

# --- MONTA REQUEST ÚNICO (75 origens + 46 alvos = 121 pontos) ---
all_locations = origens_locs + alvos_locs
all_codes = origens_codes + alvos_codes

# Junta todas as localizações: [75 origens + 46 destinos]
all_locations = origens_locs + alvos_locs

# Índices: origens = 0 a 74, destinos = 75 a 120
sources = list(range(0, len(origens_codes)))        # [0, 1, ..., 74]
destinations = list(range(len(origens_codes), len(origens_codes) + len(alvos_codes)))  # [75, 76, ..., 120]

print(f"Origens: {len(sources)} índices")
print(f"Destinos: {len(destinations)} índices")
print(f"Total de rotas solicitadas: {len(sources)} × {len(destinations)} = {len(sources)*len(destinations)}")

# --- REQUEST COM sources + destinations ---
profile = 'driving-car'
body = {
    "locations": all_locations,
    "sources": sources,           # ← SÓ calcular a partir desses
    "destinations": destinations, # ← SÓ para esses destinos
    "metrics": ["distance", "duration"],
    "units": "m"
}

try:
    print("Enviando request com sources/destinations...")
    res = client_ors.request(f'/v2/matrix/{profile}/json', {}, post_json=body)
    dist_matrix = res['distances']    # 75 × 46
    time_matrix = res['durations']    # 75 × 46
    print("Request concluído! Matriz 75×46 recebida.")
except Exception as e:
    print(f"Erro: {e}")
    raise

# --- EXTRAI RESULTADOS ---
C_dist = {}
C_time = {}

for i, orig_code in enumerate(origens_codes):
    for j, dest_code in enumerate(alvos_codes):
        dist_m = dist_matrix[i][j]
        time_s = time_matrix[i][j]
        
        if dist_m is not None and dist_m > 0:
            C_dist[(orig_code, dest_code)] = dist_m
            C_time[(orig_code, dest_code)] = time_s
        else:
            C_dist[(orig_code, dest_code)] = np.inf
            C_time[(orig_code, dest_code)] = np.inf

print(f"Extraídas {len(C_dist)} rotas válidas")

# --- GERA CSV FINAL ---
rows = []
for (orig, dest), dist_m in C_dist.items():
    if dist_m == np.inf or dist_m <= 0:
        continue
    dist_km = round(dist_m / 1000, 2)
    custo = round(dist_km * 2.50, 2)
    rows.append([orig, dest, dist_km, custo])

rotas_df = pd.DataFrame(rows, columns=['COD_ORIGEM', 'COD_DESTINO', 'DISTANCIA_KM', 'CUSTO_DESLOCAMENTO'])
rotas_df = rotas_df.sort_values(['COD_ORIGEM', 'COD_DESTINO']).reset_index(drop=True)
rotas_df.to_csv('rotas_75x46_FONTES_DESTINOS.csv', sep=';', index=False)

print(f"\nCSV salvo: {len(rotas_df)} rotas válidas")
print(f"Cobertura: {len(rotas_df)/(75*46)*100:.2f}%")

# --- SALVA DICIONÁRIOS ---
with open('C_dist_sources_dest.pkl', 'wb') as f:
    pickle.dump((C_dist, C_time), f)
print("Dicionários salvos!")

Origens: 75 índices
Destinos: 46 índices
Total de rotas solicitadas: 75 × 46 = 3450
Enviando request com sources/destinations...
Request concluído! Matriz 75×46 recebida.
Extraídas 3450 rotas válidas

CSV salvo: 3404 rotas válidas
Cobertura: 98.67%
Dicionários salvos!


In [9]:
# Parâmetros - Dados de demografia e infraestrutura
DM_max = 60.0  # km (máxima distância aceitável)
C_km = 2.50     # R$/km (custo por km rodado)
F = 4.0         # visitas/ano por paciente (frequência)
P_j = dict(zip(
    hospitais_df['CD_MUN'],
    (hospitais_df['Total'] > 0).astype(int)
))        # 1 se j é um município provedor de serviços com capacidade (fixo)

TO = 0.82 # Taxa de ocupação média esperada (82%)
DIAS_ANO = 365 # Nº de dias no ano

In [10]:
# Parâmetros - Mínimos
F_min = 2           # nº mínimo de centros de saúde

In [24]:
N_DESTINOS_POR_ORIGEM = 23 # nº de destinos possíveis
# Agrupa por origem, ordena pela distância (menor primeiro) e pega os N primeiros
rotas_df_filtrado = rotas_df.sort_values('DISTANCIA_KM').groupby('COD_ORIGEM').head(N_DESTINOS_POR_ORIGEM).reset_index(drop=True)
print(f"Total de pares (i, j) após filtragem: {len(rotas_df_filtrado)}")

Total de pares (i, j) após filtragem: 1725


In [12]:
# Parâmetros - Demandas
## recalcula com segurança (evita KeyError)
Demanda_LeitosDia_Bruta = demanda_df.set_index('CD_MUN')['total_dias_internacao'].to_dict()

## Demanda média diária e Capacidade diária
D_i_LeitosDia = {i: Demanda_LeitosDia_Bruta.get(i, 0) / DIAS_ANO for i in Demanda_LeitosDia_Bruta.keys()}
K_i_LeitosDia = {i: K_i.get(i, 0) * TO for i in Demanda_LeitosDia_Bruta.keys()}

## cria DataFrame de conferência
check_df = pd.DataFrame({
    'D_i_LeitosDia': pd.Series(D_i_LeitosDia),
    'K_i_LeitosDia': pd.Series(K_i_LeitosDia)
}).fillna(0)

check_df['diff'] = check_df['D_i_LeitosDia'] - check_df['K_i_LeitosDia']
check_df['percent_used'] = np.where(check_df['K_i_LeitosDia']>0,
                                    check_df['D_i_LeitosDia'] / check_df['K_i_LeitosDia'],
                                    np.nan)

## estatísticas gerais
print(check_df.describe())

## quantos positivos/negativos/zeros
print("counts:", (check_df['diff']>0).sum(), "municipios com déficit (D>K)")
print("percent average occupancy (median):", check_df['percent_used'].median())

## mostra os municípios com maior déficit (se houver)
print("Top deficits (maiores positivas):")
print(check_df[check_df['diff']>0].sort_values('diff', ascending=False).head(10))

## mostra os municípios com maior sobra (mais negativos)
print("Top sobras (maiores negativas):")
print(check_df.sort_values('diff').head(10))

       D_i_LeitosDia  K_i_LeitosDia         diff  percent_used
count      75.000000      75.000000    75.000000     22.000000
mean       19.654027      39.830133   -20.176107      2.922346
std        65.446084     238.228153   174.796654      4.920774
min         1.143053       0.000000 -1497.014247      0.136095
25%         3.564247       0.000000     1.600274      0.383728
50%         6.754403       0.000000     4.114990      0.505043
75%        12.604755       0.820000     7.772759      1.868508
max       554.625753    2051.640000    80.160548     16.195361
counts: 60 municipios com déficit (D>K)
percent average occupancy (median): 0.5050426802323702
Top deficits (maiores positivas):
         D_i_LeitosDia  K_i_LeitosDia       diff  percent_used
2804805     119.520548          39.36  80.160548      3.036599
2807402      26.248650           0.00  26.248650           NaN
2807105      36.082192          17.22  18.862192      2.095365
2803203      17.166575           0.00  17.166575    

In [25]:
## Parâmetros - Custos e outros
CI = 3_500_000          # custo anual médio do hospital
S = 130_000    # custo anual médio de leito médico

# --- DICIONÁRIOS DE DISTÂNCIA E CUSTO ---
D_ij = {}
CD_ij = {}
alpha_ij = {}
alpha_CD_ij = {}  # termo da FO
valid_I_J_pairs = set()

for _, row in rotas_df_filtrado.iterrows():
    i = row['COD_ORIGEM']
    j = row['COD_DESTINO']
    dist_km = row['DISTANCIA_KM']
    
    if dist_km == np.inf or dist_km <= 0:
        continue

    valid_I_J_pairs.add((i, j))
    # Distância
    D_ij[(i, j)] = dist_km
    
    # --- alpha_ij: decaimento linear ---
    if dist_km > DM_max:
        alpha = 0.0
    else:
        alpha = max(DM_max - dist_km, 0) / DM_max
    alpha_ij[(i, j)] = alpha
    
    # --- CD_ij: custo anual por paciente ---
    custo_anual = C_km * dist_km * 2 * F # ida e volta
    CD_ij[(i, j)] = custo_anual
    
    # --- alpha * CD_ij: custo efetivo ---
    alpha_CD_ij[(i, j)] = alpha * custo_anual
    
# Adicionar pares diagonais (i=j) que podem ter sido filtrados (essencial para R5)
for i in I:
    if i in J:
        if (i, i) not in valid_I_J_pairs:
            D_ij[(i, i)] = 0.0
            alpha_ij[(i, i)] = 1.0
            CD_ij[(i, i)] = 0.0
            alpha_CD_ij[(i, i)] = 0.0
            valid_I_J_pairs.add((i, i))

print(f"Total de pares (i, j) para a variável x[i, j] (excluindo diagonais): {len(valid_I_J_pairs)}")

Total de pares (i, j) para a variável x[i, j] (excluindo diagonais): 1771


In [26]:
# Modelo (Gurobi)
m = gp.Model("Localizacao_Centros_Medicos")

In [27]:
# Variáveis (Gurobi)
x = m.addVars(valid_I_J_pairs, vtype=GRB.BINARY, name="x")
z = m.addVars(J, vtype=GRB.BINARY, name="z")
w = m.addVars(J, vtype=GRB.INTEGER, name="w")
m.update()

In [28]:
# Função Objetivo
obj_centros = gp.quicksum(CI*z[j] + S*w[j] for j in J)
obj_pacientes = gp.quicksum(alpha_CD_ij[i, j] * H_i[i] * x[i, j] for i, j in valid_I_J_pairs)

m.setObjective(obj_centros + obj_pacientes, GRB.MINIMIZE)

In [29]:
# Restrições - R1, R3, R5
m.addConstrs((gp.quicksum(x[i, j] for j in J if (i, j) in valid_I_J_pairs) == 1
              for i in I), name="R1_Demanda_Servico_Satisfeita")
m.addConstrs((x[i, j] <= z[j]
              for i, j in valid_I_J_pairs), name="R3_Atribuicao_Especialidade_so_se_Aberto")
# *** REMOVIDA PARA ECONOMIZAR VARIÁVEIS 
m.addConstrs((x[j, j] == z[j]
              for j in J if (j, j) in valid_I_J_pairs), name="R5_Atendimento_Local_Obrigatorio")

{2800308: <gurobi.Constr *Awaiting Model Update*>,
 2804805: <gurobi.Constr *Awaiting Model Update*>,
 2803500: <gurobi.Constr *Awaiting Model Update*>,
 2802908: <gurobi.Constr *Awaiting Model Update*>,
 2802106: <gurobi.Constr *Awaiting Model Update*>,
 2806701: <gurobi.Constr *Awaiting Model Update*>,
 2801306: <gurobi.Constr *Awaiting Model Update*>,
 2807105: <gurobi.Constr *Awaiting Model Update*>,
 2807402: <gurobi.Constr *Awaiting Model Update*>,
 2803005: <gurobi.Constr *Awaiting Model Update*>,
 2800209: <gurobi.Constr *Awaiting Model Update*>,
 2804508: <gurobi.Constr *Awaiting Model Update*>,
 2803203: <gurobi.Constr *Awaiting Model Update*>,
 2807600: <gurobi.Constr *Awaiting Model Update*>,
 2800605: <gurobi.Constr *Awaiting Model Update*>,
 2805703: <gurobi.Constr *Awaiting Model Update*>,
 2800670: <gurobi.Constr *Awaiting Model Update*>,
 2803609: <gurobi.Constr *Awaiting Model Update*>,
 2801207: <gurobi.Constr *Awaiting Model Update*>,
 2805505: <gurobi.Constr *Await

In [30]:
# Restrição - R8
m.addConstr(gp.quicksum(z[j] for j in J) >= F_min, name="R8_Min_Centros_Medicos")

<gurobi.Constr *Awaiting Model Update*>

In [31]:
# Restrição - R10
m.addConstrs((z[j] >= P_j[j]
                  for j in J if j in P_j and P_j[j] == 1),
    name="R10_Forcar_Infraestrutura_Existente"
)

{2800308: <gurobi.Constr *Awaiting Model Update*>,
 2804805: <gurobi.Constr *Awaiting Model Update*>,
 2803500: <gurobi.Constr *Awaiting Model Update*>,
 2802908: <gurobi.Constr *Awaiting Model Update*>,
 2802106: <gurobi.Constr *Awaiting Model Update*>,
 2806701: <gurobi.Constr *Awaiting Model Update*>,
 2801306: <gurobi.Constr *Awaiting Model Update*>,
 2807105: <gurobi.Constr *Awaiting Model Update*>,
 2803005: <gurobi.Constr *Awaiting Model Update*>,
 2800209: <gurobi.Constr *Awaiting Model Update*>,
 2804508: <gurobi.Constr *Awaiting Model Update*>,
 2805703: <gurobi.Constr *Awaiting Model Update*>,
 2801207: <gurobi.Constr *Awaiting Model Update*>,
 2805406: <gurobi.Constr *Awaiting Model Update*>,
 2805901: <gurobi.Constr *Awaiting Model Update*>}

In [32]:
# R12 (Adaptada): Restrição de Capacidade AGREGADA
for j in J:
    m.addConstr(
        w[j] >= gp.quicksum(
            max(D_i_LeitosDia.get(i, 0) - K_i_LeitosDia.get(i, 0), 0) * x[i, j]
            for i in I if (i, j) in valid_I_J_pairs
        ),
        name=f"R12_Leitos_Extra_{j}"
    )

In [34]:
print(f"\nModelo redefinido com: {m.numVars} variáveis, {m.numConstrs} restrições.")
m.update()


Modelo redefinido com: 1863 variáveis, 1954 restrições.


In [35]:
# Otimização
m.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: AMD Ryzen 5 5500U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1954 rows, 1863 columns and 6924 nonzeros
Model fingerprint: 0x75592b78
Variable types: 0 continuous, 1863 integer (1817 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [3e+02, 9e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 1.789433e+08
Presolve removed 994 rows and 475 columns
Presolve time: 0.03s
Presolved: 960 rows, 1388 columns, 4440 nonzeros
Found heuristic solution: objective 1.785673e+08
Variable types: 0 continuous, 1388 integer (1342 binary)

Root relaxation: objective 1.149116e+08, 241 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl 

 9307076 5269050     cutoff   50      1.1503e+08 1.1491e+08  0.10%   1.6  370s
 9393893 5317881 1.1497e+08   53   16 1.1503e+08 1.1491e+08  0.10%   1.6  375s
 9461449 5354150 1.1502e+08   51   14 1.1503e+08 1.1491e+08  0.10%   1.6  380s
 9546186 5396822 1.1500e+08   69   13 1.1503e+08 1.1491e+08  0.10%   1.6  385s
 9605204 5423578 1.1501e+08   77   11 1.1503e+08 1.1491e+08  0.10%   1.6  390s
 9652001 5446853 1.1495e+08   69   14 1.1503e+08 1.1491e+08  0.10%   1.6  395s
 9702122 5471696 1.1499e+08   47   12 1.1503e+08 1.1491e+08  0.10%   1.6  400s
 9751805 5497533 1.1500e+08   54   18 1.1503e+08 1.1491e+08  0.10%   1.6  405s
 9815673 5530517 1.1496e+08   74   13 1.1503e+08 1.1491e+08  0.10%   1.6  410s
 9853993 5551058 1.1500e+08   56   13 1.1503e+08 1.1491e+08  0.10%   1.6  415s
 9921501 5583776     cutoff   54      1.1503e+08 1.1491e+08  0.10%   1.6  420s
 9969714 5605308     cutoff   69      1.1503e+08 1.1491e+08  0.10%   1.6  425s
 10003363 5622630 1.1493e+08   61   22 1.1503e+08 1.

 12834110 7095529 1.1492e+08   62   19 1.1503e+08 1.1491e+08  0.10%   1.6  886s
 12851576 7103810 1.1492e+08   53   17 1.1503e+08 1.1491e+08  0.10%   1.6  890s
 12889528 7120554 1.1502e+08   61   13 1.1503e+08 1.1491e+08  0.10%   1.6  896s
 12902555 7126287     cutoff   60      1.1503e+08 1.1491e+08  0.10%   1.6  900s
 12911533 7129771 1.1503e+08   73   12 1.1503e+08 1.1491e+08  0.10%   1.6  905s
 12923490 7136817     cutoff   57      1.1503e+08 1.1491e+08  0.10%   1.6  910s
 12944626 7149609 1.1502e+08  102   12 1.1503e+08 1.1491e+08  0.10%   1.6  915s
 12963536 7160268     cutoff   51      1.1503e+08 1.1491e+08  0.10%   1.6  920s
 13004789 7179687 1.1500e+08   49   15 1.1503e+08 1.1491e+08  0.10%   1.6  926s
 13028787 7192551     cutoff   58      1.1503e+08 1.1491e+08  0.10%   1.6  931s
 13034075 7195357 1.1502e+08   75   10 1.1503e+08 1.1491e+08  0.10%   1.6  936s
 13050101 7204393     cutoff   66      1.1503e+08 1.1491e+08  0.10%   1.6  941s
 13078948 7219880 1.1500e+08   74   12 1

 14537260 7980852 1.1497e+08   61   10 1.1503e+08 1.1491e+08  0.10%   1.6 1406s
 14537289 7980946 1.1499e+08   63    9 1.1503e+08 1.1491e+08  0.10%   1.6 1411s
 14547620 7988199 1.1501e+08   83   11 1.1503e+08 1.1491e+08  0.10%   1.6 1416s
 14565345 7999216 1.1500e+08   55   11 1.1503e+08 1.1491e+08  0.10%   1.6 1420s
 14595163 8018742 1.1498e+08  137    8 1.1503e+08 1.1491e+08  0.10%   1.6 1426s
 14610146 8027794 infeasible  112      1.1503e+08 1.1491e+08  0.10%   1.6 1430s
 14631610 8041369     cutoff   76      1.1503e+08 1.1491e+08  0.10%   1.6 1435s
 14661853 8061907 1.1498e+08   57   12 1.1503e+08 1.1491e+08  0.10%   1.6 1440s
 14676871 8069524 1.1503e+08   90    8 1.1503e+08 1.1491e+08  0.10%   1.6 1445s
 14676915 8069884 1.1503e+08   93    9 1.1503e+08 1.1491e+08  0.10%   1.6 1451s
 14682980 8073943 1.1500e+08  101   11 1.1503e+08 1.1491e+08  0.10%   1.6 1455s
 14711120 8092244 1.1502e+08   82   11 1.1503e+08 1.1491e+08  0.10%   1.6 1460s
 14737909 8107499 1.1499e+08  100   10 1

 16427379 9070656 1.1498e+08   98    8 1.1503e+08 1.1491e+08  0.10%   1.6 2025s
 16433022 9072230 1.1492e+08   60   14 1.1503e+08 1.1491e+08  0.10%   1.6 2031s
 16433050 9072387 1.1493e+08   62   13 1.1503e+08 1.1491e+08  0.10%   1.6 2037s
 16439816 9076191 1.1501e+08   94   10 1.1503e+08 1.1491e+08  0.10%   1.6 2041s
 16456934 9085769 1.1492e+08   66   14 1.1503e+08 1.1491e+08  0.10%   1.6 2045s
 16466798 9090718 infeasible   48      1.1503e+08 1.1491e+08  0.10%   1.6 2050s
 16503467 9109882 1.1500e+08   68   12 1.1503e+08 1.1491e+08  0.10%   1.6 2057s
 16517695 9117393 1.1500e+08   60   13 1.1503e+08 1.1491e+08  0.10%   1.6 2061s
 16537399 9127468     cutoff   77      1.1503e+08 1.1491e+08  0.10%   1.6 2065s
 16555588 9137098 1.1494e+08   54   13 1.1503e+08 1.1491e+08  0.10%   1.6 2070s
 16570450 9144818 1.1493e+08   63   15 1.1503e+08 1.1491e+08  0.10%   1.6 2075s
 16570476 9144835     cutoff   65      1.1503e+08 1.1491e+08  0.10%   1.6 2081s
 16570490 9144877 1.1491e+08   57   14 1

...

 29207048 15760748 1.1502e+08   62   11 1.1503e+08 1.1491e+08  0.10%   1.6 10702s
 29207267 15761155 1.1502e+08   61   11 1.1503e+08 1.1491e+08  0.10%   1.6 10706s
 29233061 15775528 1.1495e+08   60   17 1.1503e+08 1.1491e+08  0.10%   1.6 10717s
 29234002 15776307 1.1498e+08   61   15 1.1503e+08 1.1491e+08  0.10%   1.6 10721s
 29255192 15786881 1.1501e+08  148   11 1.1503e+08 1.1491e+08  0.10%   1.6 10731s
 29255666 15789851 1.1501e+08  149   12 1.1503e+08 1.1491e+08  0.10%   1.6 10735s
 29275499 15797833     cutoff   96      1.1503e+08 1.1491e+08  0.10%   1.6 10742s
 29278972 15800574 1.1501e+08   80    9 1.1503e+08 1.1491e+08  0.10%   1.6 10745s
 29290253 15805246 1.1496e+08   67   13 1.1503e+08 1.1491e+08  0.10%   1.6 10755s
 29292016 15806932     cutoff   57      1.1503e+08 1.1491e+08  0.10%   1.6 10760s
 29314645 15817267 1.1502e+08   72   10 1.1503e+08 1.1491e+08  0.10%   1.6 10768s
 29315647 15818486 1.1503e+08   59   16 1.1503e+08 1.1491e+08  0.10%   1.6 10770s
 29320944 158197

 29921029 16121743 1.1501e+08  127   11 1.1503e+08 1.1491e+08  0.10%   1.6 11318s
 29921736 16123475 1.1501e+08  128   10 1.1503e+08 1.1491e+08  0.10%   1.6 11322s
 29938738 16131316 1.1502e+08  100   10 1.1503e+08 1.1491e+08  0.10%   1.6 11333s
 29938979 16137748 1.1502e+08  101   10 1.1503e+08 1.1491e+08  0.10%   1.6 11337s
 29950937 16137756 1.1494e+08   57   15 1.1503e+08 1.1491e+08  0.10%   1.6 11342s
 29950950 16137765 1.1494e+08   58   15 1.1503e+08 1.1491e+08  0.10%   1.6 11349s
 29950962 16137772 1.1494e+08   59   16 1.1503e+08 1.1491e+08  0.10%   1.6 11358s
 29950977 16138227 1.1494e+08   59   16 1.1503e+08 1.1491e+08  0.10%   1.6 11362s
 29951753 16138323 1.1494e+08   58   15 1.1503e+08 1.1491e+08  0.10%   1.6 11372s
 29951901 16138846 1.1501e+08   93   11 1.1503e+08 1.1491e+08  0.10%   1.6 11376s
 29957075 16142585 1.1503e+08   56   14 1.1503e+08 1.1491e+08  0.10%   1.6 11380s
 29980477 16153976 1.1501e+08   72   11 1.1503e+08 1.1491e+08  0.10%   1.6 11389s
 29980958 161545

 30569360 16445074 1.1500e+08   56   15 1.1503e+08 1.1491e+08  0.10%   1.6 11920s
 30572352 16445259 1.1492e+08   49   19 1.1503e+08 1.1491e+08  0.10%   1.6 11929s
 30572758 16445892 1.1495e+08   50   17 1.1503e+08 1.1491e+08  0.10%   1.6 11934s
 30582049 16450840 1.1500e+08   55   11 1.1503e+08 1.1491e+08  0.10%   1.6 11935s
 30593690 16455068 1.1497e+08   67   15 1.1503e+08 1.1491e+08  0.10%   1.6 11945s
 30594138 16455253 1.1497e+08   68   13 1.1503e+08 1.1491e+08  0.10%   1.6 11957s
 30595866 16457139 1.1491e+08   52   15 1.1503e+08 1.1491e+08  0.10%   1.6 11960s
 30612648 16464076 1.1493e+08   68   13 1.1503e+08 1.1491e+08  0.10%   1.6 11968s
 30613589 16464849 1.1493e+08   69   13 1.1503e+08 1.1491e+08  0.10%   1.6 11972s
 30635339 16474417 1.1493e+08   71   17 1.1503e+08 1.1491e+08  0.10%   1.6 11982s
 30635826 16475182 1.1493e+08   72   17 1.1503e+08 1.1491e+08  0.10%   1.6 11985s
 30642406 16477482 1.1494e+08   78   21 1.1503e+08 1.1491e+08  0.10%   1.6 11994s
 30642419 164774

 31278154 16790309 1.1493e+08   64   17 1.1503e+08 1.1491e+08  0.10%   1.6 12610s
 31288300 16794142     cutoff   77      1.1503e+08 1.1491e+08  0.10%   1.6 12617s
 31289304 16794906 1.1500e+08   73   17 1.1503e+08 1.1491e+08  0.10%   1.6 12620s
 31303130 16800755     cutoff   78      1.1503e+08 1.1491e+08  0.10%   1.6 12629s
 31303716 16801276 1.1502e+08   77   13 1.1503e+08 1.1491e+08  0.10%   1.6 12634s
 31307486 16803612 1.1492e+08   70   15 1.1503e+08 1.1491e+08  0.10%   1.6 12635s
 31322791 16808972 1.1502e+08  101   10 1.1503e+08 1.1491e+08  0.10%   1.6 12645s
 31329624 16812612 1.1497e+08   79   16 1.1503e+08 1.1491e+08  0.10%   1.6 12650s
 31339611 16816245 1.1491e+08   49   17 1.1503e+08 1.1491e+08  0.10%   1.6 12659s
 31340252 16816877     cutoff   68      1.1503e+08 1.1491e+08  0.10%   1.6 12663s
 31354928 16823119 1.1500e+08   74   15 1.1503e+08 1.1491e+08  0.10%   1.6 12665s
 31357535 16823382 1.1493e+08   69   14 1.1503e+08 1.1491e+08  0.10%   1.6 12672s
 31358073 168239

GurobiError: Out of memory

In [38]:
# Extração e Análise de Resultados
# if m.status == GRB.OPTIMAL:
if m.status:
    print(f"Custo Total Anual Ótimo (Governo + Social): R${m.objVal:,.2f}")
    
    # Custo do Governo
    custo_governo = obj_centros.getValue()
    print(f"\nCusto Anual do Governo: R${custo_governo:,.2f}")
    
    # Custo Social
    custo_social = obj_pacientes.getValue()
    print(f"Custo Social de Deslocamento: R${custo_social:,.2f}")
    
    # -----------------------------------------------------
    # Análise das Decisões de Localização (Variável z_j)
    # -----------------------------------------------------
    
    centros_abertos = [j for j in J if z[j].X > 0.5]
    novos_centros = [j for j in J if z[j].X > 0.5 and P_j[j] == 0]
    centros_existentes = [j for j in J if z[j].X > 0.5 and P_j[j] == 1]
    
    print(f"\nTotal de Centros Médicos (CEMs) a serem operados: {len(centros_abertos)}")
    if novos_centros:
        print(f"- Municípios Selecionados: {novos_centros}")
    else:
        print("Nenhum novo Centro Médico foi alocado na solução ótima (pode ter atingido o limite F ou o custo é muito alto).")
    print(f"- Centros Existentes (CEAE/CEM) a serem mantidos/expandidos: {len(centros_existentes)}")
    print(f"- Novos Centros Médicos (CEMs) a serem instalados: {len(novos_centros)}")

    # -----------------------------------------------------
    # Análise das Alocações (Variável we_ej)
    # -----------------------------------------------------
            
    # Número total de especialistas a serem contratados
    alocacao_especialistas = {j: {} for j in J}
    total_especialistas_contratados = 0
    
    for j in centros_abertos:
        num_especialistas = int(round(w[j].X))
            
        if num_especialistas > 0:
            alocacao_especialistas[j] = num_especialistas
            total_especialistas_contratados += num_especialistas

    if total_especialistas_contratados > 0:
        print(f"Total de Especialistas Contratados: {total_especialistas_contratados}")
        print("\nDetalhes da Alocação:")
        
        centros_sem_contratacao = [j for j in centros_abertos if not alocacao_especialistas[j]]
        if centros_sem_contratacao:
            print("\nCentros abertos que não exigiram contratação de novos especialistas (capacidade local suficiente):")
            print(f" - {centros_sem_contratacao}")
        else:
            print("Nenhum centro não exigiu contratação de novos especialistas.")
    else:
        print("Nenhum especialista adicional foi contratado na rede. A capacidade existente (K_i) pode ser suficiente.")
            
else:
    print(f"O modelo não encontrou uma solução ótima. Status: {m.status}")
if m.status == GRB.INFEASIBLE:
      print("O modelo é inviável. Verifique as restrições de nível de serviço (F, Pq) ou os limites de capacidade.")

Custo Total Anual Ótimo (Governo + Social): R$115,030,000.00

Custo Anual do Governo: R$115,030,000.00
Custo Social de Deslocamento: R$0.00

Total de Centros Médicos (CEMs) a serem operados: 15
Nenhum novo Centro Médico foi alocado na solução ótima (pode ter atingido o limite F ou o custo é muito alto).
- Centros Existentes (CEAE/CEM) a serem mantidos/expandidos: 15
- Novos Centros Médicos (CEMs) a serem instalados: 0
Total de Especialistas Contratados: 481

Detalhes da Alocação:

Centros abertos que não exigiram contratação de novos especialistas (capacidade local suficiente):
 - [2802106, 2803005, 2805703, 2801207, 2805406]
